# Decision Tree, Random Forest and Gradient Boosting

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline 
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
%load_ext autoreload
%autoreload 2
import pipeline_tree as pipeline_tree

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
NUMERIC_FEATURES = ['age','education','num_household','num_child',
                    'partner_edu','if_own_house','if_employment_current']
CATGORICAL_FEATURES = ['if_urban','wealth_index','if_earn_more','sex_head_household', 
              'partner_ideal_child','money_decide_person']
FEATURES = NUMERIC_FEATURES + CATGORICAL_FEATURES
TARGET_LST = ['if_emo_vio', 'if_phy_vio', 'if_sex_vio', 'if_vio']


In [ ]:
# Old Features

# FEATURES = ['province','age','education', 'if_urban',
#                  'wealth_index', 'if_own_house',
#                  'if_employment', 'if_employment_current','employment_pay_method', 'if_earn_more',
#                  'partner_edu', 
#                  'num_child','sex_head_household', 'sexual_activity', 'ideal_num_child', 'partner_ideal_child', 'money_decide_person']
# NUMERIC_FEATURES = ['age','education','if_own_house','if_employment_current','partner_edu','num_child','ideal_num_child']
# CATGORICAL_FEATURES = ['province', 'if_urban',
#                  'wealth_index',
#                  'if_employment','employment_pay_method','if_earn_more', 
#                  'sex_head_household', 'sexual_activity', 'partner_ideal_child', 'money_decide_person']
# TARGET_LST = ['if_emo_vio', 'if_phy_vio', 'if_sex_vio', 'if_vio']

In [5]:
FILES = ['cleaned_data/cambodia_2014_cleaned.csv', 
         'cleaned_data/Maldives_2016_cleaned.csv', 
         'cleaned_data/Nepal_2016_cleaned.csv',
         'cleaned_data/Pakistan_2017_cleaned.csv',
         'cleaned_data/Philippines_2017_cleaned.csv']
COUNTRY_DATA = {
    'Cambodia': 'cleaned_data/cambodia_2014_cleaned.csv', 
    'Maldives': 'cleaned_data/Maldives_2016_cleaned.csv', 
    'Nepal': 'cleaned_data/Nepal_2016_cleaned.csv',
    'Pakistan': 'cleaned_data/Pakistan_2017_cleaned.csv',
    'Philippines': 'cleaned_data/Philippines_2017_cleaned.csv'   
}

In [6]:
# Initialize results data frame
RESULT_MODEL = pd.DataFrame(columns = ('country','target','model','params','f1','accuracy','precision','recall','roc_auc','model_object'))

In [8]:
country_df_lst = []
for country, file in COUNTRY_DATA.items():
    df = pipeline_tree.read_data(file)
    country_df = pipeline_tree.one_country_model_all_tree(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)
    
    country_df_lst.append(country_df)
RESULT_MODEL = pd.concat(country_df_lst)


 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Country:  Cambodia 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Country:  Maldives 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target: 

TypeError: 

In [ ]:
pd.set_option('max_colwidth',500)
RESULT_MODEL.sort_values(by=['f1'], ascending=False).head(200)

In [9]:
# Save result to csv file
RESULT_MODEL.to_csv("result_model_trees.csv")

In [10]:
country_df_lst

[     country      target                 model  \
 0   Cambodia  if_emo_vio  BalancedRandomForest   
 1   Cambodia  if_emo_vio  BalancedRandomForest   
 2   Cambodia  if_emo_vio  BalancedRandomForest   
 3   Cambodia  if_emo_vio  BalancedRandomForest   
 4   Cambodia  if_emo_vio  BalancedRandomForest   
 ..       ...         ...                   ...   
 7   Cambodia      if_vio      GradientBoosting   
 8   Cambodia      if_vio      GradientBoosting   
 9   Cambodia      if_vio      GradientBoosting   
 10  Cambodia      if_vio      GradientBoosting   
 11  Cambodia      if_vio      GradientBoosting   
 
                                                params        f1  accuracy  \
 0   {'criterion': 'gini', 'max_depth': 5, 'min_sam...  0.411602  0.603574   
 1   {'criterion': 'gini', 'max_depth': 5, 'min_sam...  0.425365  0.615974   
 2   {'criterion': 'gini', 'max_depth': 10, 'min_sa...  0.427938  0.627999   
 3   {'criterion': 'gini', 'max_depth': 10, 'min_sa...  0.425690  0.620639

In [11]:
RESULT_MODEL = pd.concat(country_df_lst)

In [12]:
RESULT_MODEL.to_csv("result_model_trees.csv")

In [13]:
country = 'Philippines'
df = pipeline_tree.read_data('cleaned_data/Philippines_2017_cleaned.csv')
country_df = pipeline_tree.one_country_model_all_tree(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)


 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Country:  Philippines 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 




KeyboardInterrupt: 

In [15]:
result_model_prev = pipeline_tree.read_data('result_model_trees.csv')
RESULT_MODEL = pd.concat([result_model_prev, country_df])
RESULT_MODEL.to_csv("result_model_trees_4.csv")

### Balanced Random Forest, Weighted Random Forest

The data is imbalanced and Random Forest doesn't work well with imbalanced data and our results show that the F1 scores and recall scores are relatively low. To alleviate the problem, I will use Balanced Random Forest and Weighted Random Forest to get better predictions. [Chen et al (2004)](https://statistics.berkeley.edu/sites/default/files/tech-reports/666.pdf)

- The regular RF classifier tends to be biased towards the majority class, so the weighted random forest places a heavier penalty on misclassifying the minority class (i.e. the minority class is given a larger weight). The weights are used in the tree induction procedure (the splits) and in the terminal nodes (class determined by weighted majority vote).

## Merged Data of Five Countries

In [23]:
NUMERIC_FEATURES = ['age','education','num_household','num_child',
                    'partner_edu','if_own_house','if_employment_current']
CATGORICAL_FEATURES = ['country', 'if_urban','wealth_index','if_earn_more','sex_head_household', 
              'partner_ideal_child','money_decide_person']
FEATURES = NUMERIC_FEATURES + CATGORICAL_FEATURES
TARGET_LST = ['if_emo_vio', 'if_phy_vio', 'if_sex_vio', 'if_vio']

In [24]:
df = pipeline_tree.read_data('cleaned_data/five_countries_data.csv')
country = 'Five_countries'
five_country_df = pipeline_tree.one_country_model_all_tree(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)


 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Country:  Five_countries 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 



 
 Target:  if_emo_vio 



 
 Target:  if_phy_vio 



 
 Target:  if_sex_vio 



 
 Target:  if_vio 




In [26]:
five_country_df.to_csv("result_model_trees_5combined.csv")

# Pipeline

In [7]:
# def split_data(features, target):
#     X_train, X_test, y_train, y_test = train_test_split(features, 
#                                                     target, 
#                                                     test_size=0.20, 
#                                                     random_state=505)
#     return X_train, X_test, y_train, y_test

In [8]:
# def preprocess_data(df, features_col, target_col, categorical_col):
#     df.dropna(subset=[target_col],inplace=True)
#     df = impute_missing_median(df, NUMERIC_FEATURES)
#     df = fill_categorical_na_vals(df)
#     features = df[features_col]
#     features = pd.get_dummies(features, columns=categorical_col)
#     target = df[target_col]
#     return features, target

In [9]:
# def impute_missing_median(df, col_lst):
#     '''
#     Impute missing values of continuous variables using the median value
#     '''
#     for col in col_lst:
#         df.loc[(df[col] == "don't know") | (df[col] == "non-numeric response") , col] = None
#         median = df[col].median()
#         df[col].fillna(median,inplace=True)
#     return df

In [10]:
# def fill_categorical_na_vals(df):
#     '''
#     Find colums and rows with missing values. Print rows, returns list of
#     columns.
#     '''
#     df = df.fillna(0)
#     return df

In [11]:
# def train_decision_tree(X_train, X_test, y_train, y_test):
#     params = {'criterion': ['gini', 'entropy'],
#                 'max_depth': [3,5,10,15],
#                 'min_samples_split': [2,5,10]}
#     grid_model = GridSearchCV(estimator=DecisionTreeClassifier(random_state=505), 
#                               param_grid=params, 
#                               cv=10,
#                               return_train_score=True,
#                               scoring=['f1', 'accuracy','precision','recall','roc_auc'],
#                               refit='f1')

#     grid_model.fit(X_train, y_train)
#     grid_result = pd.DataFrame(grid_model.cv_results_)
#     grid_result = grid_result[['params','mean_test_f1','mean_test_accuracy', 'mean_test_precision','mean_test_recall','mean_test_roc_auc']]
#     grid_result = grid_result.sort_values(by=['mean_test_f1'], ascending=False)
#     return grid_result
# #     print('Best model params: ', grid_result.loc[grid_result['mean_train_accuracy'] == max(grid_result['mean_train_accuracy'])]['params'])

In [12]:
# def train_random_forest(X_train, X_test, y_train, y_test):
#     params = {'n_estimators':[10, 100],
#               'criterion': ['gini', 'entropy'],
#               'max_depth': [3,5,10,15],
#               'min_samples_split': [2,5,10]}
#     grid_model = GridSearchCV(estimator=RandomForestClassifier(random_state=505), 
#                               param_grid=params, 
#                               cv=10,
#                               return_train_score=True,
#                               scoring=['f1', 'accuracy','precision','recall','roc_auc'],
#                               refit='f1')

#     grid_model.fit(X_train, y_train)
#     grid_result = pd.DataFrame(grid_model.cv_results_)
#     grid_result = grid_result[['params','mean_test_f1','mean_test_accuracy', 'mean_test_precision','mean_test_recall','mean_test_roc_auc']]
#     grid_result = grid_result.sort_values(by=['mean_test_f1'], ascending=False)
#     return grid_result


In [13]:
# def evaluate_test(model, X_test, y_test):
#     y_pred = model.predict(X_test)
#     plot_precision_recall_curve(model, X_test, y_test)
#     results_dict = {}
#     results_dict['f1'] = metrics.f1_score(y_test, y_pred)
#     results_dict['accuracy'] = metrics.accuracy_score(y_test, y_pred)
#     results_dict['precision'] = metrics.precision_score(y_test, y_pred)
#     results_dict['recall'] = metrics.recall_score(y_test, y_pred)
#     results_dict['roc_auc'] = metrics.roc_auc_score(y_test, y_pred)
#     plot_precision_recall_curve(model,X_test,y_test)    
#     return results_dict

In [14]:
# def plot_importances(model, X_train, n=10, title=''):
#     '''
#     Compute the relative importance of selected features in
#     the model
    
#     Inputs:
#     - model
#     - n (int): top n features, opt
#     - title (str)
#     '''
#     plt.close()
#     importances = model.feature_importances_
#     np_features = np.array(X_train.columns)
#     sorted_idx = np.argsort(importances)[len(np_features)-n:]
#     padding = np.arange(len(sorted_idx)) + 0.5
#     pl.barh(padding, importances[sorted_idx], align='center')
#     pl.yticks(padding, np_features[sorted_idx])
#     pl.xlabel("Relative Importance")
#     pl.title(title)
#     pl.show()

In [15]:
# def read_data(csv):
#     return pd.read_csv(csv)

# Cambodia

In [ ]:
# df = pipeline_tree.read_data('cleaned_data/cambodia_2014_cleaned.csv')
# country = 'Cambodia'

In [ ]:
# RESULT_MODEL = pipeline_tree.one_country_model_decision_tree(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)

In [ ]:
# RESULT_MODEL

In [ ]:
# RESULT_MODEL = pipeline_tree.one_country_model_random_forest(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)

In [ ]:
# RESULT_MODEL = pipeline_tree.one_country_model_gradient_boosting(df, country, TARGET_LST, FEATURES, CATGORICAL_FEATURES, NUMERIC_FEATURES, RESULT_MODEL)

In [ ]:
# RESULT_MODEL

In [ ]:
# target_col = 'if_emo_vio'
# features, target = preprocess_data(df, FEATURES, target_col, CATGORICAL_FEATURES)
# X_train, X_test, y_train, y_test = split_data(features, target)
# grid_result= train_decision_tree(X_train, X_test, y_train, y_test)

In [ ]:
# grid_result.columns

In [ ]:
# pd.set_option('max_colwidth',500)

In [ ]:
# def one_country_model_decision_tree(df, TARGET_LST, FEATURES, CATGORICAL_FEATURES):
#     '''
#     Use decision tree to model data of one country
#     '''
#     for target_col in TARGET_LST:
#         print("\n \n Target: ", target_col, "\n\n")
#         features, target = preprocess_data(df, FEATURES, target_col, CATGORICAL_FEATURES)
#         X_train, X_test, y_train, y_test = split_data(features, target)
#         grid_result= train_decision_tree(X_train, X_test, y_train, y_test)
#         for params in grid_result['params'][:3]:
#             print('Params: ', params)
#             good_model = DecisionTreeClassifier(**params).fit(X_train, y_train)
#             result_dict = evaluate_test(good_model, X_test, y_test)
#             print(result_dict)
#             plot_importances(good_model, X_train, n=10, title= target_col + str(params))

In [ ]:
# one_country_model_decision_tree(df, TARGET_LST, FEATURES, CATGORICAL_FEATURES)

### Adjust Threshold
Since the recall score is low, we could try to adjust the classification threshold to improve the recall score.

In [ ]:
thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]

In [ ]:
pred_prob = model.predict_proba(X_test)[:,1]
thresholds_result = pd.DataFrame(columns = ['Threshold', 'Precision', 'Recall'])
for thres in thresholds:
    y_pred = (pred_prob > thres)
    precision = precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    thres_dict = {}
    thres_dict['Threshold'] = thres
    thres_dict['Precision'] = precision
    thres_dict['Recall'] = recall
    thresholds_result = thresholds_result.append(thres_dict, ignore_index=True)   

[Reference: Decision Tree Ensembles- Bagging and Boosting](https://towardsdatascience.com/decision-tree-ensembles-bagging-and-boosting-266a8ba60fd9)